In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import GroupKFold
from choice_learn.datasets import load_swissmetro
from choice_learn.data import ChoiceDataset
from choice_learn.models import ResLogit, TasteNet, LearningMNL, RUMnet

# Deshabilitar GPU para consistencia
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# Cargar datos
train_df = pd.read_csv("https://raw.githubusercontent.com/BSifringer/EnhancedDCM/refs/heads/master/ready_example/swissmetro_paper/swissmetro_train.dat", sep="\t")
test_df = pd.read_csv("https://raw.githubusercontent.com/BSifringer/EnhancedDCM/refs/heads/master/ready_example/swissmetro_paper/swissmetro_test.dat", sep="\t")

In [ ]:
def preprocess_data(df):
    df = df.loc[(df.CAR_AV == 1) & (df.SM_AV == 1) & (df.TRAIN_AV == 1)]
    df[["TRAIN_TT", "SM_TT", "CAR_TT"]] /= 100.0
    df[["TRAIN_HE", "SM_HE"]] /= 100.0
    df["train_free_ticket"] = df.GA.apply(lambda x: 1 if x == 1 else 0)
    df["sm_free_ticket"] = df.GA.apply(lambda x: 1 if x == 1 else 0)
    df["TRAIN_travel_cost"] = (df["TRAIN_CO"] * (1 - df["train_free_ticket"])) / 100
    df["SM_travel_cost"] = (df["SM_CO"] * (1 - df["sm_free_ticket"])) / 100
    df["CAR_travel_cost"] = df["CAR_CO"] / 100
    df["CHOICE"] -= 1
    # Convertir todas las columnas relevantes a float32
    for col in df.select_dtypes(include=["float64", "int64"]).columns:
        df[col] = df[col].astype(np.float32)
    return df

train_df = preprocess_data(train_df)
test_df = preprocess_data(test_df)

# Crear ChoiceDataset
def create_dataset(df):
    return ChoiceDataset.from_single_wide_df(
        df=df,
        choices_column="CHOICE",
        items_id=["TRAIN", "SM", "CAR"],
        shared_features_columns=["GA", "AGE", "SM_SEATS", "LUGGAGE", "PURPOSE", "FIRST", "TICKET", "WHO", "MALE", "INCOME", "ORIGIN", "DEST"],
        items_features_suffixes=["TT", "travel_cost", "HE"],
        choice_format="items_index"
    )

full_dataset = create_dataset(pd.concat([train_df, test_df]))
customers_id = np.concatenate([train_df["ID"].values, test_df["ID"].values])


In [6]:
# Parámetros comunes
n_items = 3
n_items_features = 3  # TT, travel_cost, HE
n_shared_features = 12  # GA, AGE, SM_SEATS, LUGGAGE, PURPOSE, FIRST, TICKET, WHO, MALE, INCOME, ORIGIN, DEST

# Configuración ResLogit
reslogit_args = {
    "intercept": "item",
    "optimizer": "SGD",
    "lr": 1e-6,
    "epochs": 50
}

# Configuración TasteNet
tastenet_args = {
    "taste_net_layers": [],
    "taste_net_activation": "relu",
    "items_features_by_choice_parametrization": [
        [-1., "-exp", "-exp"],  # Características para el tren (3 características)
        [-1., "-exp", "-exp"],  # Características para el metro suizo (3 características)
        [-1., "-exp", 0.]       # Características para el coche (3 características)
    ],
    "optimizer": "Adam",
    "epochs": 40,
    "lr": 0.001,
    "batch_size": 32
}

# Configuración LearningMNL
learningmnl_args = {
    "optimizer": "Adam",
    "lr": 0.005,
    "nn_features": ['PURPOSE', 'FIRST', 'TICKET', 'WHO', 'MALE', 'INCOME', 'ORIGIN', 'DEST'],
    "nn_layers_widths": [200],
    "epochs": 50,
    "batch_size": 32
}


# Configuración RUMnet
rumnet_args = {
    "num_products_features": n_items_features,
    "num_customer_features": n_shared_features,
    "width_eps_x": 20,
    "depth_eps_x": 5,
    "heterogeneity_x": 10,
    "width_eps_z": 20,
    "depth_eps_z": 5,
    "heterogeneity_z": 10,
    "width_u": 20,
    "depth_u": 5,
    "optimizer": "Adam",
    "lr": 0.0002,
    "epochs": 50,
    "batch_size": 32,
    "tol": 0,
}


In [7]:
def train_model(model_name, train_data, test_data):
    if model_name == "ResLogit":
        model = ResLogit(n_layers=4, **reslogit_args)
        model.instantiate(n_items=n_items, n_shared_features=n_shared_features, n_items_features=n_items_features)
    elif model_name == "TasteNet":
        model = TasteNet(**tastenet_args)
    elif model_name == "LearningMNL":
        model = LearningMNL(**learningmnl_args)
        model.add_shared_coefficient(feature_name="TT", items_indexes=[0, 1, 2])
        model.add_shared_coefficient(feature_name="travel_cost", items_indexes=[0, 1, 2])
        model.add_shared_coefficient(feature_name="HE", items_indexes=[0, 1])
        model.add_shared_coefficient(feature_name="GA", items_indexes=[0, 1])
        model.add_shared_coefficient(feature_name="AGE", items_indexes=[0])
        model.add_shared_coefficient(feature_name="LUGGAGE", items_indexes=[2])
        model.add_shared_coefficient(feature_name="SM_SEATS", items_indexes=[1])
        model.add_coefficients(feature_name="intercept", items_indexes=[1, 2])
    elif model_name == "RUMnet":
        model = RUMnet(**rumnet_args)
        model.instantiate()
    
    history = model.fit(train_data, val_dataset=test_data)
    train_loss = model.evaluate(train_data)
    test_loss = model.evaluate(test_data)
    return history, train_loss, test_loss

In [ ]:
gkf = GroupKFold(n_splits=5)
results = {model: {"train_loss": [], "test_loss": []} for model in ["ResLogit", "TasteNet", "LearningMNL", "RUMnet"]}

for fold, (train_idx, test_idx) in enumerate(gkf.split(full_dataset, groups=customers_id)):
    print(f"\n=== Fold {fold + 1}/5 ===")
    train_data = full_dataset[train_idx]
    test_data = full_dataset[test_idx]
    
    for model_name in results.keys():
        print(f"\nEntrenando {model_name}...")
        history, train_loss, test_loss = train_model(model_name, train_data, test_data)
        results[model_name]["train_loss"].append(train_loss)
        results[model_name]["test_loss"].append(test_loss)

In [ ]:
# Graficar resultados
plt.figure(figsize=(12, 6))
colors = {"ResLogit": "blue", "TasteNet": "green", "LearningMNL": "red", "RUMnet": "purple"}

for model_name in results.keys():
    avg_test_loss = np.mean(results[model_name]["test_loss"])
    plt.bar(model_name, avg_test_loss, color=colors[model_name], label=model_name)

plt.xlabel("Modelo")
plt.ylabel("Pérdida promedio en prueba")
plt.title("Comparación de modelos en Swissmetro Dataset")
plt.legend()
plt.show()

# Imprimir resultados numéricos
print("\nResultados detallados:")
for model_name in results.keys():
    avg_train = np.mean(results[model_name]["train_loss"])
    avg_test = np.mean(results[model_name]["test_loss"])
    print(f"{model_name}:")
    print(f"  - Train Loss: {avg_train:.4f}")
    print(f"  - Test Loss:  {avg_test:.4f}\n")